In [ ]:
import requests
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST #
from torch.utils.data import DataLoader

In [ ]:
last_update_id=None

In [ ]:
def send_telegram_message(message):
    TOKEN='' #
    chat_id='' #
    url=f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
    response=requests.get(url)
    print(f"Sent message: {message}, Response: {response.json()}") 

In [ ]:
def request_next_command():
    send_telegram_message("Please enter the next command (rerun, stop):")
    command=get_last_command(30) 
    while command not in ["rerun","stop"]:
        if command:send_telegram_message("Invalid command. Please enter 'rerun' to rerun or 'stop' to stop training.")
        command=get_last_command(30)  
    return command

In [ ]:
def get_last_command(timeout=30):
    global last_update_id
    TOKEN ='' #
    url=f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    start_time = time.time()
    print("Waiting for command...")
    while time.time()-start_time < timeout:
        response = requests.get(url).json()
        print(f"Checking for updates, Response: {response}") 
        try:
            updates = response['result']
            if updates:
                for update in reversed(updates):
                    update_id = update['update_id']
                    if 'message' in update and (last_update_id is None or update_id > last_update_id):
                        last_update_id = update_id
                        command = update['message']['text'].lower()
                        print(f"Received command: {command}")
                        return command
        except (IndexError, KeyError) as e:
            print(f"Error while processing updates: {e}") 
        time.sleep(1)  
    return ""

In [ ]:
#replace this with your model code
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),nn.ReLU(),nn.Linear(512, 10))
    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
train_dataset= MNIST(root='./data', train=True, transform=transform, download=True)
train_loader= DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
model =SimpleNet()
criterion= nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)

def train_model(num_epochs,train_loader,model,criterion,optimizer):
    model.train()
    for epoch in range(num_epochs):
        for _, (data,targets) in enumerate(train_loader):
            outputs= model(data)
            loss=criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return loss.item()



In [ ]:
#bot log
while True:
    send_telegram_message("Model training has started.")
    final_loss=train_model(1,train_loader,model,criterion,optimizer)
    send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")
    command=request_next_command()
    if command=="stop":
        send_telegram_message("Training stopped by user command.")
        break
    elif command=="rerun":
        send_telegram_message("Rerunning the training.")
        continue